In [1]:
from xgboost import XGBClassifier
# read data
from sklearn.model_selection import train_test_split
import pandas as pd
import sklearn

# fit xgboost on an imbalanced classification dataset
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier

# Generate and plot a synthetic imbalanced classification dataset
from collections import Counter
from sklearn.datasets import make_classification
from matplotlib import pyplot
from numpy import where

In [22]:
nasa = pd.read_csv('nasa.csv')  

In [20]:
all_features = [
'Absolute Magnitude',
'Est Dia in KM(min)',
'Est Dia in KM(max)',
'Epoch Date Close Approach',   # read up on this
'Relative Velocity km per sec',
'Miss Dist.(kilometers)',
#'Orbiting Body', # Only Earth
'Orbit Uncertainity', # Uncertainty of what the orbit looks like
'Minimum Orbit Intersection', # Nearest dist. to earth during orbit
'Jupiter Tisserand Invariant',
'Epoch Osculation',
#'Eccentricity', # How streched is the elipsis. This can be given by Minimum Orbit Intersection and Semi Major Axis
'Semi Major Axis', # Half length of longest radius of orbit
'Inclination', # Angle of orbital plane compared to earths orbital plane
'Asc Node Longitude', # read up on this
'Orbital Period', # Time to complete orbit in days
'Perihelion Distance', # Closest distance to sun during orbit
'Perihelion Arg', # What is the angle compared to earths plane, when NEO is closest to sun
'Aphelion Dist', # Farthest distance to sun during orbit (AU)
#'Perihelion Time', # When is closest distance to sun achieved (as Julian date). 
#'Mean Anomaly', # What is the angle at a given time?
#'Mean Motion', # Average angular speed - could be given by inclination, I think
#'Equinox' # This is a constant in the data
'Hazardous'
]

In [10]:
# explore distribution between hazardous and non-hazardous
nasa.groupby('Hazardous').apply(len)

/var/folders/hg/l2wdlfb97db6m9qjl00xzxg40000gn/T/ipykernel_5843/2535362033.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nasa.groupby('Hazardous').apply(len)


Hazardous
False    3932
True      755
dtype: int64

In [9]:
755/3932

0.1920142421159715

In [23]:


# Filtering the DataFrame based on the list of column names
nasa = nasa[all_features]

In [26]:
nasa

,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Epoch Date Close Approach,Relative Velocity km per sec,Miss Dist.(kilometers),Orbit Uncertainity,Minimum Orbit Intersection,Jupiter Tisserand Invariant,Epoch Osculation,Semi Major Axis,Inclination,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Hazardous
0,21.600,0.127220,0.284472,788947200000,6.115834,6.275369e+07,5,0.025282,4.634,2458000.5,1.407011,6.025981,314.373913,609.599786,0.808259,57.257470,2.005764,True
1,21.300,0.146068,0.326618,788947200000,18.113985,5.729815e+07,3,0.186935,5.457,2458000.5,1.107776,28.412996,136.717242,425.869294,0.718200,313.091975,1.497352,False
2,20.300,0.231502,0.517654,789552000000,7.590711,7.622912e+06,0,0.043058,4.557,2458000.5,1.458824,4.237961,259.475979,643.580228,0.950791,248.415038,1.966857,True
3,27.400,0.008801,0.019681,790156800000,11.173874,4.268362e+07,6,0.005512,5.093,2458000.5,1.255903,7.905894,57.173266,514.082140,0.983902,18.707701,1.527904,False
4,21.600,0.127220,0.284472,790156800000,9.840831,6.101082e+07,1,0.034798,5.154,2458000.5,1.225615,16.793382,84.629307,495.597821,0.967687,158.263596,1.483543,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4682,23.900,0.044112,0.098637,1473318000000,22.154265,6.187511e+06,8,0.019777,5.156,2457637.5,1.161429,39.880491,164.183305,457.179984,0.741558,276.395697,1.581299,False
4683,28.200,0.006089,0.013616,1473318000000,3.225150,9.677324e+05,6,0.006451,5.742,2458000.5,1.075134,5.360249,345.225230,407.185767,0.996434,42.111064,1.153835,False
4684,22.700,0.076658,0.171412,1473318000000,7.191642,9.126775e+06,6,0.059972,4.410,2458000.5,1.528234,4.405467,37.026468,690.054279,0.965760,274.692712,2.090708,False
4685,21.800,0.116026,0.259442,1473318000000,11.352090,3.900908e+07,5,0.177510,4.477,2458000.5,1.486600,21.080244,163.802910,662.048343,1.185467,180.346090,1.787733,False


In [15]:
nasa["Orbiting Body"].unique() # only earth as orbiting body, and equinox is constant

array(['Earth'], dtype=object)

In [42]:
X = nasa[['Absolute Magnitude','Est Dia in KM(min)','Est Dia in KM(max)',
'Epoch Date Close Approach',  
'Relative Velocity km per sec',
'Miss Dist.(kilometers)',
'Orbit Uncertainity', 
'Minimum Orbit Intersection', 
'Jupiter Tisserand Invariant',
'Epoch Osculation',
'Semi Major Axis',
'Inclination', 
'Asc Node Longitude', 
'Orbital Period', 
'Perihelion Distance', 
'Perihelion Arg',
'Aphelion Dist']]


X = nasa[['Est Dia in KM(min)','Est Dia in KM(max)',
'Epoch Date Close Approach',  
'Relative Velocity km per sec',
'Miss Dist.(kilometers)',
'Orbit Uncertainity', 
'Jupiter Tisserand Invariant',
'Epoch Osculation',
'Semi Major Axis',
'Inclination', 
'Asc Node Longitude', 
'Orbital Period', 
'Perihelion Distance', 
'Perihelion Arg',
'Aphelion Dist']]

y = nasa["Hazardous"]

In [43]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [46]:
# define model https://machinelearningmastery.com/xgboost-for-imbalanced-classification/

model = XGBClassifier(scale_pos_weight=6.2)


# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.5f' % mean(scores))

Mean ROC AUC: 0.97039


In [44]:
model.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [45]:
# Get feature importances
feature_importances = model.feature_importances_

# Create a DataFrame to display feature importances
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Display feature importances
print(feature_importance_df)


                         Feature  Importance
0             Est Dia in KM(min)    0.374241
12           Perihelion Distance    0.146752
9                    Inclination    0.077677
13                Perihelion Arg    0.073558
5             Orbit Uncertainity    0.046500
4         Miss Dist.(kilometers)    0.043798
14                 Aphelion Dist    0.041867
6    Jupiter Tisserand Invariant    0.039813
8                Semi Major Axis    0.038909
2      Epoch Date Close Approach    0.032664
10            Asc Node Longitude    0.030689
3   Relative Velocity km per sec    0.028904
7               Epoch Osculation    0.024629
1             Est Dia in KM(max)    0.000000
11                Orbital Period    0.000000
